<a href="https://colab.research.google.com/github/Natural-State/agol-data-workflows/blob/master/code/Colab%20notebooks/10_hydroSHEDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extract hydroBASINS and hydroRIVERS from the hydroSHEDS product

## Import gee and authenticate

In [ ]:
import ee

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

## Input arguments for data extraction

In [ ]:
# Area of interest
aoi = ee.FeatureCollection("projects/ns-agol-rs-data/assets/LLBN")
aoi_name = "LLBN"

# Basin level (3 to 12)
basin_list = ["03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]

# GEE layer ID
layer_dict = {
  "03": "RS_022",
  "04": "RS_023",
  "05": "RS_024",
  "06": "RS_025",
  "07": "RS_026",
  "08": "RS_027",
  "09": "RS_028",
  "10": "RS_029",
  "11": "RS_030",
  "12": "RS_031"
}

## Define functions and process hydroATLAS data

In [ ]:
# Function to extract basins for specific level
def extract_basin(basin_id):
  feat_coll_id = "WWF/HydroATLAS/v1/Basins/level" + basin_id
  basin_atlas = ee.FeatureCollection(feat_coll_id).filterBounds(aoi)
  field_list = basin_atlas.first().propertyNames().getInfo()
  field_list = [element for element in field_list if not element.startswith(('snw', 'pnv', "glc"))]
  basin_atlas = basin_atlas.select(field_list)
  return(basin_atlas)


In [ ]:
# Map over basin list
basins = list(map(extract_basin, basin_list))

# Check
print(basins)
field_names = basins[0].first().propertyNames()
print(field_names.getInfo())

## Export data

In [ ]:
for i in range(len(basins)):
  layer_name = layer_dict[basin_list[i]]
  output_name = f"{layer_name}_{aoi_name}"
  print(output_name)

  task = ee.batch.Export.table.toDrive(
    collection = basins[i],
    description = "EXPORT IMAGE TO DRIVE",
    fileFormat = "SHP",
    fileNamePrefix = output_name,
    folder = "GEE_exports"
     )

  task.start()
  print("STARTED TASK ", i+1)

In [ ]:
tasks = ee.batch.Task.list()
for task in tasks[0:len(basins)]:
  task_id = task.status()['id']
  task_state = task.status()['state']
  print(task_id, task_state)